In [3]:
import pandas as pd
import numpy as np

In [4]:
#readin csv
df= pd.read_csv("/Users/nikitadvortsevoy/Desktop/Foundations of CSS/Final project/Dataset/MC_Fake_dataset.csv")

df.head()

,news_id,title,url,publish_date,source,text,labels,n_tweets,n_retweets,n_replies,n_users,tweet_ids,retweet_ids,reply_ids,user_ids,retweet_relations,reply_relations,data_name
0,Claim-JuneFC18,WHO said artemisia plant cures Covid-19.,NaN,2020-05-05,NaN,"«Il n’y a rien de nouveau, juste un effet gros...",1,1,8,43,43,1258265925658259458,"1258266492573044736,1258268929732104192,125828...","1258265928321699840,1258266668821872640,125826...","1096021910972977152,529538368,1258273607278624...",1258266492573044736-1258265925658259458-722618...,1258265928321699840-1258265925658259458-284967...,FakeCovidClaimFiltered
1,Claim-JuneFC41,“There was no real scientific basis for belie...,NaN,2020-05-04,NaN,"More Info, Fox News host Laura Ingraham speaks...",1,4,1,5148,3759,"1258329173141094401,1258514570945032192,128065...",1280680704116998144,"1280654816352624640,1280655159643660288,128065...","795107478157586432,866860504203046914,14261352...",NaN,1280654816352624640-1280654150955565056-133715...,FakeCovidClaimFiltered
2,Claim-JuneFC46,Joe Biden has “written a letter of apology” fo...,NaN,2020-05-03,NaN,"More Info, President Donald Trump speaks durin...",1,3,57,12,66,"1257361723872419841,1257633359351746561,125770...","1257361909428486145,1257362106883719171,125736...","1257362357795336193,1257362452246859778,125736...","826295010542772224,1211394495654162433,1080666...",1257361909428486145-1257361723872419841-794136...,1257362357795336193-1257361723872419841-832319...,FakeCovidClaimFiltered
3,Claim-JuneFC47,Did US House Speaker Pelosi 'Hold Back' Corona...,NaN,2020-03-11,NaN,"Hirsch, Laura and Kevin Breuninger.\xa0“Trump ...",1,10,2,0,12,"1237873947925856256,1238121200989151232,123823...","1240854409371926528,1240858669895438336",NaN,"1137466967269498880,18526305,72153358413834240...",1240854409371926528-1240661167724470273-113746...,NaN,FakeCovidClaimFiltered
4,Claim-JuneFC55,"“A few hours ago, China officially announced ...",NaN,2020-04-03,NaN,Tanto en Facebook como en Twitter se ha vuelto...,1,5,15,13,23,"1239254388377563143,1239378517697212422,124000...","1239258123950600192,1239379736222150662,123956...","1240026703860314112,1240060236838379521,124006...","1069892389425180672,828922917174071297,1079227...",1239258123950600192-1239254388377563143-633421...,1240026703860314112-1239378517697212422-664343...,FakeCovidClaimFiltered


In [5]:
#CHECKING THE DATA

#number of rows and columns
print(df.shape)

#columns
print("\nColumns:", df.columns.tolist())

#checking unique fields
print(df.nunique(dropna=True).head(20))

#checking for Nan's
print(df.isna().sum())

#checking for Nan's in % 
print((df.isna().mean() * 100).round(2))

#data looks suitable for our purposes 

(28335, 18)

Columns: ['news_id', 'title', 'url', 'publish_date', 'source', 'text', 'labels', 'n_tweets', 'n_retweets', 'n_replies', 'n_users', 'tweet_ids', 'retweet_ids', 'reply_ids', 'user_ids', 'retweet_relations', 'reply_relations', 'data_name']
news_id              28335
title                28326
url                  21649
publish_date         12072
source                1352
text                 27750
labels                   2
n_tweets               932
n_retweets            1322
n_replies              946
n_users               1832
tweet_ids            28008
retweet_ids          24608
reply_ids            18818
user_ids             28049
retweet_relations    24521
reply_relations      18758
data_name               11
dtype: int64
news_id                  0
title                    0
url                   6661
publish_date         10439
source               14393
text                   202
labels                   0
n_tweets                 0
n_retweets               0
n_replie

In [6]:
# STEP 1 ()
# dividing Tweet IDs from user ID's according to -  retweet_relations: retweet relations indicated by a list of tokens {tweet_ID_A}-{tweet_ID_B}-{user_ID of tweet A}-{user_ID of tweet B} denoting A retweets

# tacking one row
ret_relations_str = df.loc[0, "retweet_relations"]

#spliting tokens to get rid of the gaps
ret_tokens = ret_relations_str.split(",")

#getting rid of gaps
ret_clean_tokens = []
for t in ret_tokens:
    ret_clean_tokens.append(t.strip())

ret_tokens = ret_clean_tokens

#splitting ID's and checking it on several rows 

ret_splitted = ret_tokens[0].split("-")
print(len(ret_splitted))
print(ret_splitted)

#nice, it's working
# tried on one row to check if it works 



4
['1258266492573044736', '1258265925658259458', '722618840371372037', '284967242']


In [7]:
#STEP 2
#Creating edge list for retweets - who(user A) retweeted or replied to whom (user B)

retweet_edges = []

for ret_tok in ret_tokens:
    parts = ret_tok.split("-")
    ret_user_a = parts[2]
    ret_user_b = parts[3]
    retweet_edges.append((ret_user_a, ret_user_b))

#checking
print(len(retweet_edges))
print(retweet_edges[:5])

8
[('722618840371372037', '284967242'), ('2309085185', '284967242'), ('1062422785249722368', '284967242'), ('1258273607278624769', '284967242'), ('529538368', '284967242')]


In [8]:
#STEP 2.1
#Doing the same with repplies - structure is the same reply_relations: reply relations indicated by a list of tokens {tweet_ID_A}-{tweet_ID_B}-{user_ID of tweet A}-{user_ID of tweet B} denoting A replies B

# tacking one row
rep_relations_str = df.loc[0, "reply_relations"]

#spliting tokens to get rid of the gaps
rep_tokens = rep_relations_str.split(",")

#getting rid of gaps
rep_clean_tokens = []
for t in rep_tokens:
    rep_clean_tokens.append(t.strip())

rep_tokens = rep_clean_tokens

#splitting ID's and checking it on several rows 

rep_splitted = rep_tokens[0].split("-")

#creating edge list 
reply_edges = []

for tok in rep_tokens:
    parts = tok.split("-")
    rep_user_a = parts[2]
    rep_user_b = parts[3]
    reply_edges.append((rep_user_a, rep_user_b))


#checking
print(len(reply_edges))
print(reply_edges[:5])

43
[('284967242', '284967242'), ('932575358569930752', '284967242'), ('989212314204360705', '284967242'), ('1234160752757477377', '284967242'), ('1163015951417319425', '284967242')]


In [9]:
#STEP 3
# Building edge list for retweets for news_id

all_retweet_edges = []

# preparing retweets relations
for index in range(len(df)) :  
    news_id = df.loc[index, "news_id"]
    ret_relations_str = df.loc[index, "retweet_relations"]
    if pd.isna(ret_relations_str):  # to avoid mistake with Nan
        continue

    ret_tokens = ret_relations_str.split(",")

    ret_clean_tokens = []
    for t in ret_tokens:
        ret_clean_tokens.append(t.strip())

    ret_tokens = ret_clean_tokens

    # now edges
    for ret_token in ret_tokens:
        parts = ret_token.split("-")
        ret_user_a = parts[2]
        ret_user_b = parts[3]
        all_retweet_edges.append((news_id, ret_user_a, ret_user_b, "retweet"))

#check
print(len(all_retweet_edges))
print(all_retweet_edges[:5])


2833074
[('Claim-JuneFC18', '722618840371372037', '284967242', 'retweet'), ('Claim-JuneFC18', '2309085185', '284967242', 'retweet'), ('Claim-JuneFC18', '1062422785249722368', '284967242', 'retweet'), ('Claim-JuneFC18', '1258273607278624769', '284967242', 'retweet'), ('Claim-JuneFC18', '529538368', '284967242', 'retweet')]


In [10]:
# STEP 3.2
# Building edge list for replies for news_id

all_reply_edges = []

for index in range(len(df)):
    news_id = df.loc[index, "news_id"]

    rep_relations_str = df.loc[index, "reply_relations"]
    if pd.isna(rep_relations_str):
        continue

    rep_tokens = rep_relations_str.split(",")

    rep_clean_tokens = []
    for t in rep_tokens:
        rep_clean_tokens.append(t.strip())
    rep_tokens = rep_clean_tokens

    for rep_token in rep_tokens:
        parts = rep_token.split("-")
        rep_user_a = parts[2]
        rep_user_b = parts[3]
        all_reply_edges.append((news_id, rep_user_a, rep_user_b, "reply"))



#check
print(len(all_reply_edges))
print(all_reply_edges[:5])


1629038
[('Claim-JuneFC18', '284967242', '284967242', 'reply'), ('Claim-JuneFC18', '932575358569930752', '284967242', 'reply'), ('Claim-JuneFC18', '989212314204360705', '284967242', 'reply'), ('Claim-JuneFC18', '1234160752757477377', '284967242', 'reply'), ('Claim-JuneFC18', '1163015951417319425', '284967242', 'reply')]


In [11]:
#STEP 3.3
# Unite to have all edges 

all_edges = all_retweet_edges + all_reply_edges

print("All_edges (retweet + reply)", len(all_edges))

All_edges (retweet + reply) 4462112


In [12]:
#creating a df with edges so it be easier to work with it

edges_df = pd.DataFrame(all_edges,columns=["news_id", "source_user", "destination_user", "interaction_kind"])

edges_df.head()

,news_id,source_user,destination_user,interaction_kind
0,Claim-JuneFC18,722618840371372037,284967242,retweet
1,Claim-JuneFC18,2309085185,284967242,retweet
2,Claim-JuneFC18,1062422785249722368,284967242,retweet
3,Claim-JuneFC18,1258273607278624769,284967242,retweet
4,Claim-JuneFC18,529538368,284967242,retweet


In [13]:
# choosing one event to test 
sample = df.loc[0,"news_id"]
event_edges = edges_df[edges_df["news_id"] == sample]

In [14]:
# building undirected interaction graph by Wattz

import networkx as nx

graph = nx.Graph()
for _, row in event_edges.iterrows():
    src_usr = row["source_user"]
    dst_usr = row["destination_user"]
    graph.add_edge(src_usr, dst_usr)

#checking numbers
print("nodes:", graph.number_of_nodes())
print("edges:", graph.number_of_edges())

nodes: 43
edges: 47


In [15]:
#calculating clustering coefficient C 

C = nx.average_clustering(graph)
print("C - average clustering for one event:", C)

C - average clustering for one event: 0.13962432915921288


In [20]:
#calculating avarage shortest path length 

# finding all connected components in indirect graph
components = list(nx.connected_components(graph)) 

#LCC - largest connected component, max - choosing the largest component by nodes  
lcc_nodes = max(components, key=len) 

#tacking only the LCC nodes and edges between them 
graph_for_lcc = graph.subgraph(lcc_nodes)

#calculation L - ASPL
L = nx.average_shortest_path_length(graph_for_lcc)

print("Average shortest path length:",L)



Average shortest path length: 2.036544850498339


In [ ]:
# For now we computed and build a user interaction graph for one event (one news_id)
# We have NODES(source users and destination users) and EDGES (built on retweets and replies)

# And now we have two metrics from "small world":
# C - clustering - how local the network is
# L - (avarage shortest path length) - the avarage number of steps between user (but only for LCC of the graph)

# Not sure if we need to use LCC, don't quiet understand this part from lit



In [ ]:
#building a list for all news_id

all_news = df["news_id"].unique()



#checking numbers
print(len(all_news))
print(all_news[:5])

#all rows included

28335
['Claim-JuneFC18' 'Claim-JuneFC41' 'Claim-JuneFC46' 'Claim-JuneFC47'
 'Claim-JuneFC55']


In [69]:
#building graph for events and computing C and L for it

results = []

# group edges once for faster access (no full-table scan each loop) - ADDED TO EXPEDITE CALCULATION PROCESS
grouped = edges_df.groupby("news_id")

# Limit for very large networks - - ADDED TO EXPEDITE CALCULATION PROCESS - need to be added in limitations
MAX_LCC_N = 3000

# ********* C BLOCK STARTS HERE *********

for nid in all_news: # all dataset
    #event_edges = edges_df[edges_df["news_id"] == nid] #filter edges for one event, one news_id

    try: #ADDED TO EXPEDITE CALCULATION PROCESS
        event_edges = grouped.get_group(nid)
    except KeyError:
        continue

    graph = nx.Graph() #building undirected graph

    #adding edges src_usr->dst_usr
    for _, row in event_edges.iterrows():
        src_usr = row["source_user"]
        dst_usr = row["destination_user"]
        graph.add_edge(src_usr, dst_usr)

    #exluding small graphs with small number of nodes and ages - we can change numbers and see how the results would change
    if graph.number_of_nodes() < 3 or graph.number_of_edges() < 2:
        continue

    #results for C
    C = nx.average_clustering(graph)

    # ********* L BLOCK STARTS HERE *********
    components = list(nx.connected_components(graph))
    lcc_nodes = max(components, key=len)
    graph_lcc = graph.subgraph(lcc_nodes).copy()

    #exluding small graphs whith smal number of nodes - we can change numbers and see how the results would change
    if graph_lcc.number_of_nodes() < 3:
        continue

    # CHANGED/ADDED: use MAX_LCC_N before calculating L (saves a lot of time)
    N = graph_lcc.number_of_nodes()
    if N > MAX_LCC_N:
        continue

    #results for L
    L = nx.average_shortest_path_length(graph_lcc)

    #saving results
    results.append((nid, C, L))


print("computed events:", len(results))
print("first 5 results:", results[:5])


# ********** CONCLUSION **********
# This block computes the two core small-world diagnostics for each event (news_id) by building an undirected user interaction graph from retweet/reply edges.
# It calculates the clustering coefficient C on the full event graph and the average shortest path length L on the event’s largest connected component (LCC) to ensure paths are defined. 
# The output is a list of per-event results (news_id, C, L) ready for later comparison and null-model normalization.


computed events: 22438
first 5 results: [('Claim-JuneFC18', 0.13962432915921288, 2.036544850498339), ('Claim-JuneFC46', 0.03230769230769231, 1.9976621858562245), ('Claim-JuneFC47', 0.0, 1.3333333333333333), ('Claim-JuneFC55', 0.08902691511387163, 1.7777777777777777), ('Claim-JuneFC149', 0.0, 1.8666666666666667)]


In [70]:
#b building a random/null graph and calculating C_random and L_random for each news_id

#generating R random graphs per event

R = 20 # number of random graphs for event, we can change this number in order to see results for more random graphs
null_results = []

grouped = edges_df.groupby("news_id") # added grouop by news_id to expedite the process!

# Limit for very large networks - - ADDED TO EXPEDITE CALCULATION PROCESS - need to be added in limitations
MAX_LCC_N = 3000  # CHANGED/ADDED: define once and actually use it below

# ********* C BLOCK STARTS HERE *********

for nid in all_news: #if number is here then we limiting our events 

    try:
        event_edges = grouped.get_group(nid)  # GROUPBY: get edges for this news_id without scanning the whole table
    except KeyError:
        continue

    # event_edges = edges_df[edges_df["news_id"] == nid]

    graph = nx.Graph()
    for _, row in event_edges.iterrows():
        src_usr = row["source_user"]
        dst_usr = row["destination_user"]
        graph.add_edge(src_usr, dst_usr)

    #exluding small graphs with small number of nodes and ages - we can change numbers and see how the results would change
    if graph.number_of_nodes() < 3 or graph.number_of_edges() < 2:  # CHANGED: moved outside the edge-adding loop
        continue  # CHANGED: now correctly skips the whole event (nid)

    #results for C
    C = nx.average_clustering(graph)  # CHANGED: moved outside the edge-adding loop

    # ********* L BLOCK STARTS HERE *********

    components = list(nx.connected_components(graph))  # CHANGED: moved outside the edge-adding loop
    lcc_nodes = max(components, key=len)               # CHANGED: moved outside the edge-adding loop
    graph_lcc = graph.subgraph(lcc_nodes).copy()       # CHANGED: moved outside the edge-adding loop

    if graph_lcc.number_of_nodes() < 3:  # CHANGED: moved outside the edge-adding loop
        continue

    #matching the null model to the LCC size
    N = graph_lcc.number_of_nodes()  # CHANGED: moved before L to allow early skipping
    E = graph_lcc.number_of_edges()  # CHANGED: moved before L to allow early skipping

    if N > MAX_LCC_N:  # CHANGED: use the limit before calculating L (saves time)
        continue #exluding too big events to save time on calculating 

    #results for L
    L = nx.average_shortest_path_length(graph_lcc)  # CHANGED: moved AFTER the size check

    # building random/null graphs and calculating C_random and L_random 

    sum_C_rand = 0.0
    sum_L_rand = 0.0
    count_L = 0  # some random graphs might be too disconnected for L

    for _ in range(R):
        rand_g = nx.gnm_random_graph(N, E)  # undirected random graph with same N and E
        sum_C_rand += nx.average_clustering(rand_g)

        rand_components = list(nx.connected_components(rand_g))
        rand_lcc_nodes = max(rand_components, key=len)
        rand_lcc = rand_g.subgraph(rand_lcc_nodes).copy()

        #exluding small graphs whith smal number of nodes - we can change numbers and see how the results would change
        if rand_lcc.number_of_nodes() < 3:
            continue

        sum_L_rand += nx.average_shortest_path_length(rand_lcc)
        #how many succesfull calculations for L for random graph
        count_L += 1

    # in case if all random graphs were too small for L then just skip this event
    if count_L == 0:
        continue

    C_random = sum_C_rand / R
    L_random = sum_L_rand / count_L # average only over valid random L calculations 

    #saving results
    null_results.append((nid, C, L, C_random, L_random))

print("processed events:", len(null_results))  # CHANGED: print the correct list
print("first 5 rows:", null_results[:5])       # CHANGED: print the correct list


# ********** CONCLUSION **********
# In this block, we calculate small-world metrics for each news event (news_id) using the user interaction graph.
# We calculate clustering (C) on the full event graph and average shortest path length (L) on the largest connected component (LCC).
# To judge these values properly, we also build R random/null graphs with the same number of nodes (N) and edges (E) as the event’s LCC.
# For these random graphs, we calculate C_random and L_random and take the average to get a baseline.
# This gives us (C, L) and (C_random, L_random) for each event, so later we can normalize the results and compare Fake vs Real.


processed events: 22438
first 5 rows: [('Claim-JuneFC18', 0.13962432915921288, 2.036544850498339, 0.029745293466223694, 4.072758848243067), ('Claim-JuneFC46', 0.03230769230769231, 1.9976621858562245, 0.02065778853914447, 4.649718102354574), ('Claim-JuneFC47', 0.0, 1.3333333333333333, 0.0, 1.3333333333333328), ('Claim-JuneFC55', 0.08902691511387163, 1.7777777777777777, 0.1135185185185185, 2.2190079365079365), ('Claim-JuneFC149', 0.0, 1.8666666666666667, 0.15972222222222218, 1.7916666666666667)]


In [72]:


# 1) Convert null_results into a DataFrame
null_df = pd.DataFrame(
    null_results,
    columns=["news_id", "C", "L", "C_random", "L_random"]
)


# print("null_df shape:", null_df.shape)
# print(null_df.head())

# 2) Build a lookup table for labels (one label per news_id)
labels_df = df[["news_id", "labels"]].drop_duplicates()

# print("labels_df shape:", labels_df.shape)

# 3) Merge labels into null_df
final_df = null_df.merge(labels_df, on="news_id", how="left")

# print("final_df shape:", final_df.shape)
# print("missing labels:", final_df["labels"].isna().sum())
# print(final_df.head())

# 4) Calculate normalized ratios and sigma
final_df["C_ratio"] = final_df["C"] / final_df["C_random"]
final_df["L_ratio"] = final_df["L"] / final_df["L_random"]
final_df["sigma"] = final_df["C_ratio"] / final_df["L_ratio"]

print(final_df[["news_id", "labels", "C_ratio", "L_ratio", "sigma"]].head())


           news_id  labels   C_ratio   L_ratio     sigma
0   Claim-JuneFC18       1  4.693997  0.500041  9.387232
1   Claim-JuneFC46       1  1.563947  0.429631  3.640212
2   Claim-JuneFC47       1       NaN  1.000000       NaN
3   Claim-JuneFC55       1  0.784250  0.801159  0.978895
4  Claim-JuneFC149       1  0.000000  1.041860  0.000000


In [ ]:
# MASTER TABLE (1 row = 1 news_id) for the whole team
# Exports a CSV with renamed columns (short explanations in brackets)

import numpy as np

# 1) Select columns and rename them with short explanations
master_df = final_df[
    ["news_id", "labels",
     "C", "L", "C_random", "L_random",
     "C_ratio", "L_ratio", "sigma"]
].copy()

master_df = master_df.rename(columns={
    "news_id": "news_id",
    "labels": "labels (veracity: 0/1)",
    "C": "C (clustering)",
    "L": "L (avg shortest path on LCC)",
    "C_random": "C_random (clustering, null)",
    "L_random": "L_random (avg shortest path on LCC, null)",
    "C_ratio": "C_ratio (C / C_random)",
    "L_ratio": "L_ratio (L / L_random)",
    "sigma": "sigma ((C/C_random) / (L/L_random))"
})

# 2) Replace inf values with NaN to keep the CSV clean
master_df = master_df.replace([np.inf, -np.inf], np.nan)

# 3) Save to CSV 
output_file = "master_small_world_table.csv"
master_df.to_csv(output_file, index=False)

print(output_file)



Saved: master_small_world_table.csv
Shape: (22438, 9)
           news_id  labels (veracity: 0/1)  C (clustering)  \
0   Claim-JuneFC18                       1        0.139624   
1   Claim-JuneFC46                       1        0.032308   
2   Claim-JuneFC47                       1        0.000000   
3   Claim-JuneFC55                       1        0.089027   
4  Claim-JuneFC149                       1        0.000000   

   L (avg shortest path on LCC)  C_random (clustering, null)  \
0                      2.036545                     0.029745   
1                      1.997662                     0.020658   
2                      1.333333                     0.000000   
3                      1.777778                     0.113519   
4                      1.866667                     0.159722   

   L_random (avg shortest path on LCC, null)  C_ratio (C / C_random)  \
0                                   4.072759                4.693997   
1                                   4.649718

In [75]:
# SUMMARY TABLE (Fake/Real)
# Exports medians and quartiles for sigma, C_ratio, L_ratio by label

import pandas as pd
import numpy as np

# Use the same table you saved as master_df (or build it again from final_df)
# master_df should contain: labels (veracity: 0/1), C_ratio, L_ratio, sigma

# 1) Clean values: remove rows with NaN/inf in key metrics
clean_master = master_df.copy()
clean_master = clean_master.replace([np.inf, -np.inf], np.nan)
clean_master = clean_master.dropna(subset=[
    "C_ratio (C / C_random)",
    "L_ratio (L / L_random)",
    "sigma ((C/C_random) / (L/L_random))",
    "labels (veracity: 0/1)"
])

# 2) Helper function to calculate q25, median, q75
def q25(x): return x.quantile(0.25)
def q75(x): return x.quantile(0.75)

# 3) Group by label and calculate summary stats
summary_df = clean_master.groupby("labels (veracity: 0/1)").agg(
    count=("sigma ((C/C_random) / (L/L_random))", "count"),

    median_sigma=("sigma ((C/C_random) / (L/L_random))", "median"),
    q25_sigma=("sigma ((C/C_random) / (L/L_random))", q25),
    q75_sigma=("sigma ((C/C_random) / (L/L_random))", q75),

    median_C_ratio=("C_ratio (C / C_random)", "median"),
    q25_C_ratio=("C_ratio (C / C_random)", q25),
    q75_C_ratio=("C_ratio (C / C_random)", q75),

    median_L_ratio=("L_ratio (L / L_random)", "median"),
    q25_L_ratio=("L_ratio (L / L_random)", q25),
    q75_L_ratio=("L_ratio (L / L_random)", q75),
).reset_index()

# 4) Rename label column to a clearer name
summary_df = summary_df.rename(columns={
    "labels (veracity: 0/1)": "label (veracity: 0/1)"
})

# 5) Save to CSV
output_file = "summary_small_world_table.csv"
summary_df.to_csv(output_file, index=False)

print("Saved:", output_file)
print("Shape:", summary_df.shape)
print(summary_df)


Saved: summary_small_world_table.csv
Shape: (2, 11)
   label (veracity: 0/1)  count  median_sigma  q25_sigma  q75_sigma  \
0                      0  17909           0.0        0.0        0.0   
1                      1   2873           0.0        0.0        0.0   

   median_C_ratio  q25_C_ratio  q75_C_ratio  median_L_ratio  q25_L_ratio  \
0             0.0          0.0          0.0        0.613536     0.445371   
1             0.0          0.0          0.0        0.687274     0.473445   

   q75_L_ratio  
0     0.844730  
1     0.909091  


In [76]:


# Use final_df (the table with C, L, C_random, L_random, labels, and ratios if you calculated them)
df_check = final_df.copy()

print("Rows in final_df:", len(df_check))
print("\nLabel distribution:")
print(df_check["labels"].value_counts(dropna=False))

# 1) How many zeros in C and C_random
c_zero = (df_check["C"] == 0).sum()
cr_zero = (df_check["C_random"] == 0).sum()

print("\nC == 0 count:", c_zero, "| share:", round(c_zero / len(df_check), 4))
print("C_random == 0 count:", cr_zero, "| share:", round(cr_zero / len(df_check), 4))

# 2) Basic descriptive stats
print("\nDescriptive stats (overall):")
print(df_check[["C", "C_random", "L", "L_random"]].describe())

# 3) Check zeros by label
print("\nShare of C == 0 by label:")
print(df_check.groupby("labels")["C"].apply(lambda s: (s == 0).mean()))

print("\nShare of C_random == 0 by label:")
print(df_check.groupby("labels")["C_random"].apply(lambda s: (s == 0).mean()))

# 4) If ratios/sigma exist, check NaN/inf and stats
if "C_ratio" in df_check.columns:
    tmp = df_check.replace([np.inf, -np.inf], np.nan)

    print("\nNaN counts (ratios/sigma):")
    for col in ["C_ratio", "L_ratio", "sigma"]:
        if col in tmp.columns:
            print(col, "NaN:", tmp[col].isna().sum())

    print("\nDescriptive stats for ratios (finite only):")
    cols = [c for c in ["C_ratio", "L_ratio", "sigma"] if c in tmp.columns]
    print(tmp[cols].describe())


Rows in final_df: 22438

Label distribution:
labels
0    19184
1     3254
Name: count, dtype: int64

C == 0 count: 18349 | share: 0.8178
C_random == 0 count: 1656 | share: 0.0738

Descriptive stats (overall):
                  C      C_random             L      L_random
count  22438.000000  22438.000000  22438.000000  22438.000000
mean       0.005687      0.068194      1.997569      3.587285
std        0.027265      0.083582      0.544872      2.009403
min        0.000000      0.000000      1.000000      1.000000
25%        0.000000      0.011575      1.714286      1.907708
50%        0.000000      0.043928      1.913043      3.044042
75%        0.000000      0.104630      2.032016      4.849674
max        1.000000      1.000000      7.296797     10.581803

Share of C == 0 by label:
labels
0    0.818442
1    0.813768
Name: C, dtype: float64

Share of C_random == 0 by label:
labels
0    0.066462
1    0.117087
Name: C_random, dtype: float64

NaN counts (ratios/sigma):
C_ratio NaN: 1656
L

In [77]:
# Compare C and L between labels (0/1) using simple summary stats

import numpy as np

tmp = final_df.copy()
tmp = tmp.replace([np.inf, -np.inf], np.nan)

# Keep only rows where C and L exist
tmp = tmp.dropna(subset=["labels", "C", "L"])

print("Rows used:", len(tmp))
print("\nCounts by label:")
print(tmp["labels"].value_counts())

print("\nC (clustering) by label:")
print(tmp.groupby("labels")["C"].agg(["count", "mean", "median", "min", "max"]))

print("\nL (avg shortest path on LCC) by label:")
print(tmp.groupby("labels")["L"].agg(["count", "mean", "median", "min", "max"]))


Rows used: 22438

Counts by label:
labels
0    19184
1     3254
Name: count, dtype: int64

C (clustering) by label:
        count      mean  median  min  max
labels                                   
0       19184  0.005380     0.0  0.0  1.0
1        3254  0.007501     0.0  0.0  1.0

L (avg shortest path on LCC) by label:
        count      mean    median  min       max
labels                                          
0       19184  2.011596  1.916667  1.0  7.296797
1        3254  1.914873  1.866667  1.0  7.161180


In [80]:
#PERMUTATION TEST

# SETTINGS
P = 3000          # number of permutations (start 2000–5000)
SEED = 42
REAL = 0
FAKE = 1

rng = np.random.default_rng(SEED)

def permutation_test_median_diff(x_fake, x_real, P, rng):
    """
    Permutation test for diff in medians: median(fake) - median(real)
    Keeps group sizes fixed.
    """
    n_fake = len(x_fake)
    combined = np.concatenate([x_fake, x_real])

    obs = np.median(x_fake) - np.median(x_real)

    diffs = []
    for _ in range(P):
        perm = rng.permutation(combined)
        perm_fake = perm[:n_fake]
        perm_real = perm[n_fake:]
        diffs.append(np.median(perm_fake) - np.median(perm_real))

    diffs = np.array(diffs)
    p_value = np.mean(np.abs(diffs) >= np.abs(obs))  # two-sided p-value
    return obs, p_value

# Choose metrics to compare (recommend L_ratio first; sigma can be zero-heavy)
metrics = ["L_ratio", "L"]  # you can add "C_ratio" or "sigma" if you want

for metric in metrics:
    # Prepare clean data for this metric
    tmp = final_df.replace([np.inf, -np.inf], np.nan).dropna(subset=["labels", metric])

    x_real = tmp.loc[tmp["labels"] == REAL, metric].values
    x_fake = tmp.loc[tmp["labels"] == FAKE, metric].values

    print("\n==============================")
    print("METRIC:", metric)
    print("n_real:", len(x_real), "| n_fake:", len(x_fake))

    if len(x_real) == 0 or len(x_fake) == 0:
        print("Skip: one group is empty after cleaning.")
        continue

    med_real = np.median(x_real)
    med_fake = np.median(x_fake)
    obs_diff = med_fake - med_real

    print("Median Real:", med_real)
    print("Median Fake:", med_fake)
    print("Observed diff (Fake - Real):", obs_diff)

    obs, p_val = permutation_test_median_diff(x_fake, x_real, P, rng)
    print("Permutation test p-value (two-sided):", p_val)



METRIC: L_ratio
n_real: 19184 | n_fake: 3254
Median Real: 0.6433944339973632
Median Fake: 0.7525643123223118
Observed diff (Fake - Real): 0.10916987832494862
Permutation test p-value (two-sided): 0.0

METRIC: L
n_real: 19184 | n_fake: 3254
Median Real: 1.9166666666666667
Median Fake: 1.8666666666666667
Observed diff (Fake - Real): -0.050000000000000044
Permutation test p-value (two-sided): 0.0


In [83]:

# Settings
P = 5000
SEED = 42
REAL = 0
FAKE = 1

rng = np.random.default_rng(SEED)

# 1) Clean data for C_ratio (keep only valid baseline cases)
tmp = final_df.replace([np.inf, -np.inf], np.nan).copy()
tmp["C_ratio"] = tmp["C"] / tmp["C_random"]
tmp = tmp.dropna(subset=["labels", "C_ratio", "C_random"])
tmp = tmp[tmp["C_random"] > 0]

x_real = tmp.loc[tmp["labels"] == REAL, "C_ratio"].values
x_fake = tmp.loc[tmp["labels"] == FAKE, "C_ratio"].values

print("n_real:", len(x_real), "| n_fake:", len(x_fake))

# 2) Observed difference in shares where C_ratio > 1
share_real = np.mean(x_real > 1)
share_fake = np.mean(x_fake > 1)
obs_diff = share_fake - share_real

print("Share Real (C_ratio > 1):", share_real)
print("Share Fake (C_ratio > 1):", share_fake)
print("Observed diff (Fake - Real):", obs_diff)

# 3) Permutation test for the share difference (two-sided)
combined = np.concatenate([x_fake, x_real])
n_fake = len(x_fake)

perm_diffs = []
for _ in range(P):
    perm = rng.permutation(combined)
    perm_fake = perm[:n_fake]
    perm_real = perm[n_fake:]
    perm_diffs.append(np.mean(perm_fake > 1) - np.mean(perm_real > 1))

perm_diffs = np.array(perm_diffs)
p_value = np.mean(np.abs(perm_diffs) >= np.abs(obs_diff))

print("Permutation p-value (two-sided):", p_value)
print("p-value resolution approx:", 1 / P)


n_real: 17909 | n_fake: 2873
Share Real (C_ratio > 1): 0.1455692668490703
Share Fake (C_ratio > 1): 0.15001740341106856
Observed diff (Fake - Real): 0.004448136561998273
Permutation p-value (two-sided): 0.531
p-value resolution approx: 0.0002


In [103]:

# ----------------------------
# Settings
# ----------------------------
REAL = 0
FAKE = 1
SEED = 42
B = 3000   # bootstrap iterations (can use 2000–5000)
P = 5000   # permutation iterations
rng = np.random.default_rng(SEED)

# ----------------------------
# Helper: bootstrap CI for median difference
# ----------------------------
def bootstrap_ci_median_diff(x_fake, x_real, B, rng):
    diffs = []
    n_f = len(x_fake)
    n_r = len(x_real)
    for _ in range(B):
        samp_f = rng.choice(x_fake, size=n_f, replace=True)
        samp_r = rng.choice(x_real, size=n_r, replace=True)
        diffs.append(np.median(samp_f) - np.median(samp_r))
    diffs = np.array(diffs)
    return np.percentile(diffs, 2.5), np.percentile(diffs, 97.5)

# ----------------------------
# Helper: permutation test for median difference
# ----------------------------
def perm_p_median_diff(x_fake, x_real, P, rng):
    n_f = len(x_fake)
    combined = np.concatenate([x_fake, x_real])
    obs = np.median(x_fake) - np.median(x_real)
    diffs = []
    for _ in range(P):
        perm = rng.permutation(combined)
        diffs.append(np.median(perm[:n_f]) - np.median(perm[n_f:]))
    diffs = np.array(diffs)
    p = np.mean(np.abs(diffs) >= np.abs(obs))
    return obs, p

# ----------------------------
# Helper: bootstrap CI for share difference (share_fake - share_real)
# ----------------------------
def bootstrap_ci_share_diff(x_fake_bool, x_real_bool, B, rng):
    diffs = []
    n_f = len(x_fake_bool)
    n_r = len(x_real_bool)
    for _ in range(B):
        samp_f = rng.choice(x_fake_bool, size=n_f, replace=True)
        samp_r = rng.choice(x_real_bool, size=n_r, replace=True)
        diffs.append(np.mean(samp_f) - np.mean(samp_r))
    diffs = np.array(diffs)
    return np.percentile(diffs, 2.5), np.percentile(diffs, 97.5)

# ----------------------------
# Helper: permutation test for share difference (share_fake - share_real)
# ----------------------------
def perm_p_share_diff(x_fake_bool, x_real_bool, P, rng):
    n_f = len(x_fake_bool)
    combined = np.concatenate([x_fake_bool, x_real_bool]).astype(int)
    obs = np.mean(x_fake_bool) - np.mean(x_real_bool)
    diffs = []
    for _ in range(P):
        perm = rng.permutation(combined)
        diffs.append(np.mean(perm[:n_f]) - np.mean(perm[n_f:]))
    diffs = np.array(diffs)
    p = np.mean(np.abs(diffs) >= np.abs(obs))
    return obs, p

# ----------------------------
# Prepare clean data once
# ----------------------------
tmp = final_df.replace([np.inf, -np.inf], np.nan).copy()

# Ensure ratios exist (if already exist, these overwrite consistently)
tmp["C_ratio"] = tmp["C"] / tmp["C_random"]
tmp["L_ratio"] = tmp["L"] / tmp["L_random"]
tmp["sigma"] = tmp["C_ratio"] / tmp["L_ratio"]

# ----------------------------
# A) L_ratio: median diff + CI + perm test
# ----------------------------
tmp_Lr = tmp.dropna(subset=["labels", "L_ratio"])
x_real = tmp_Lr.loc[tmp_Lr["labels"] == REAL, "L_ratio"].values
x_fake = tmp_Lr.loc[tmp_Lr["labels"] == FAKE, "L_ratio"].values

med_real = np.median(x_real)
med_fake = np.median(x_fake)
ci_lo, ci_hi = bootstrap_ci_median_diff(x_fake, x_real, B, rng)
obs, pval = perm_p_median_diff(x_fake, x_real, P, rng)

print("\n=== L_ratio (L/L_random): median(Fake) - median(Real) ===")
print("n_real:", len(x_real), "| n_fake:", len(x_fake))
print("median Real:", med_real)
print("median Fake:", med_fake)
print("diff (Fake-Real):", obs)
print("bootstrap 95% CI:", (ci_lo, ci_hi))
print("permutation p-value:", pval, "| resolution ~", 1/P)

# SAVE results for plotting later (no re-calculation needed)
effects = {}  # add once (if not already created)

effects["L_ratio (Fake − Real)"] = {
    "obs": float(obs),
    "ci_low": float(ci_lo),
    "ci_high": float(ci_hi),
    "p": float(pval)
}


# ----------------------------
# B) L (on LCC): median diff + CI + perm test
# ----------------------------
tmp_L = tmp.dropna(subset=["labels", "L"])
x_real = tmp_L.loc[tmp_L["labels"] == REAL, "L"].values
x_fake = tmp_L.loc[tmp_L["labels"] == FAKE, "L"].values

med_real = np.median(x_real)
med_fake = np.median(x_fake)
ci_lo, ci_hi = bootstrap_ci_median_diff(x_fake, x_real, B, rng)
obs, pval = perm_p_median_diff(x_fake, x_real, P, rng)

print("\n=== L (avg shortest path on LCC): median(Fake) - median(Real) ===")
print("n_real:", len(x_real), "| n_fake:", len(x_fake))
print("median Real:", med_real)
print("median Fake:", med_fake)
print("diff (Fake-Real):", obs)
print("bootstrap 95% CI:", (ci_lo, ci_hi))
print("permutation p-value:", pval, "| resolution ~", 1/P)

effects["L (Fake − Real)"] = {
    "obs": float(obs),
    "ci_low": float(ci_lo),
    "ci_high": float(ci_hi),
    "p": float(pval)
}


# ----------------------------
# C) Clustering signal as a SHARE: share(C_ratio > 1) diff + CI + perm test
# (cleaning: require C_random > 0 so the ratio is defined)
# ----------------------------
tmp_Cr = tmp.dropna(subset=["labels", "C_ratio", "C_random"])
tmp_Cr = tmp_Cr[tmp_Cr["C_random"] > 0]

real_bool = (tmp_Cr.loc[tmp_Cr["labels"] == REAL, "C_ratio"].values > 1)
fake_bool = (tmp_Cr.loc[tmp_Cr["labels"] == FAKE, "C_ratio"].values > 1)

share_real = np.mean(real_bool)
share_fake = np.mean(fake_bool)

ci_lo, ci_hi = bootstrap_ci_share_diff(fake_bool, real_bool, B, rng)
obs, pval = perm_p_share_diff(fake_bool, real_bool, P, rng)

print("\n=== Share(C_ratio > 1): share(Fake) - share(Real) ===")
print("n_real:", len(real_bool), "| n_fake:", len(fake_bool))
print("share Real:", share_real)
print("share Fake:", share_fake)
print("diff (Fake-Real):", obs)
print("bootstrap 95% CI:", (ci_lo, ci_hi))
print("permutation p-value:", pval, "| resolution ~", 1/P)

effects["Share(C_ratio > 1) (Fake − Real)"] = {
    "obs": float(obs),
    "ci_low": float(ci_lo),
    "ci_high": float(ci_hi),
    "p": float(pval)
}




=== L_ratio (L/L_random): median(Fake) - median(Real) ===
n_real: 19184 | n_fake: 3254
median Real: 0.6433944339973632
median Fake: 0.7525643123223118
diff (Fake-Real): 0.10916987832494862
bootstrap 95% CI: (np.float64(0.08811908171464863), np.float64(0.13123227249914957))
permutation p-value: 0.0 | resolution ~ 0.0002

=== L (avg shortest path on LCC): median(Fake) - median(Real) ===
n_real: 19184 | n_fake: 3254
median Real: 1.9166666666666667
median Fake: 1.8666666666666667
diff (Fake-Real): -0.050000000000000044
bootstrap 95% CI: (np.float64(-0.06593406593406592), np.float64(-0.04166666666666674))
permutation p-value: 0.0 | resolution ~ 0.0002

=== Share(C_ratio > 1): share(Fake) - share(Real) ===
n_real: 17909 | n_fake: 2873
share Real: 0.1455692668490703
share Fake: 0.15001740341106856
diff (Fake-Real): 0.004448136561998273
bootstrap 95% CI: (np.float64(-0.008980330656841812), np.float64(0.01876591332477411))
permutation p-value: 0.5266 | resolution ~ 0.0002
